In [4]:
import pandas as pd
import numpy as np
import re
import csv

In [5]:
data_req = pd.read_excel("File for analytics team - contingent labor requests_MASKED_cleaned.xlsx", "Requests")
data_tax = pd.read_excel("File for analytics team - contingent labor requests_MASKED_cleaned.xlsx", "Taxonomy")

In [6]:
data_req.head()

,Masked Worker ID,Job Posting Title,Description,CIO
0,2444,Database Administrator - III,"Plan, coordinate, and administer database syst...",NaN
1,2470,Software Systems Engineer - III,Works on problems of moderate and varied compl...,NaN
2,2477,Systems Administrator - III,"Performs complex analysis, acquisition, instal...",NaN
3,2488,Software Systems Engineer - III,Works on problems of moderate and varied compl...,Faheem
4,2492,Software Systems Engineer - III,Works on problems of moderate and varied compl...,NaN


In [7]:
data_tax.head()

,#,Source,Type,L1,L2,L3,L4,L2.1,L3.1,L4.1,All Skills
0,1.0,VZ IT taxonomy,NaN,Security,.NET,Development,.Net,.net,development,.net,net
1,2.0,VZ IT taxonomy,NaN,Development,.NET,Application Development,.Net Core,.net,application development,.net core,net core
2,3.0,VZ IT taxonomy,NaN,Network,Geospatial information systems,Mapping tools,3GIS,geospatial information systems,mapping tools,3gis,3gis
3,4.0,VZ IT taxonomy,NaN,Network,Standards,Wireless Standards,3GPP,standards,wireless standards,3gpp,3gpp
4,5.0,VZ IT taxonomy,NaN,Development,IBM,Mainframe,AbendAid,ibm,mainframe,abendaid,abendaid


In [8]:
skills = data_tax["All Skills"]
desc = data_req["Description"]
#skills = skills.apply(lambda x : x.lower())

In [9]:
def check_skill(description, skill):
    # Convert to lowercases
    s = description.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    res = s.find(' ' + str(skill) + ' ')
    
    return res
    
    # Break sentence in the token, remove empty tokens
    #tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    #ngrams = zip(*[tokens[i:] for i in range(n)])
    #return [" ".join(ngram) for ngram in ngrams]

In [10]:
ex_li = [] 
for k in range(desc.shape[0]) : 
    ctr = 0
    li = []
    for j in range(skills.shape[0]) :
        num = check_skill(desc[k], skills[j])
        if num != -1 :
            li.append(skills[j])
    ex_li.append(li)  

In [11]:
len(ex_li)

1823

In [12]:
cols = data_req.columns.tolist()
cols
cols_new = ['CIO', 'Masked Worker ID', 'Job Posting Title']
final = data_req[cols_new]
final.shape

(1823, 3)

In [24]:
final['Skills'] = ex_li
final['Job Description'] = data_req['Description']
type(final)

pandas.core.frame.DataFrame

In [37]:
all_skill = [j for i in ex_li for j in i]

32284

In [15]:
uni_skill = set(all_skill)
final['Masked Worker ID'][2]

2477

In [17]:
my_dict = {}
for i in uni_skill : 
    my_dict[i] = []

In [18]:
for item in uni_skill : 
    lt = []
    for index, skill_set in enumerate(final['Skills']) : 
        if (item in skill_set) : 
            lt.append(final['Masked Worker ID'][index])
            my_dict[item] = lt

In [36]:
with open('output.csv', 'w', newline = '') as output:
    writer = csv.writer(output)
    for key, value in my_dict.items():
        writer.writerow([key, value])

In [81]:
final.to_excel("Final.xlsx")